In [156]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib as plt

#Import cranial database 
df = pd.read_csv('../../datasources/fdb/postcranial.csv')
#This shuffles the rows.
df = df.sample(frac=1)

#Drop categorical data. We'll focus only on measurement data.
del df['Item']
del df['ID']
del df['ContNum']
del df['FDN']
del df['Pop']
del df['PopSex']
del df['BirthYear']
del df['Age'] #Probably really important.
del df['Comments']
del df['DB']

#I don't know what these are.
# del df['FStat']
# del df['CStat']
# del df['MStat']

rows, cols = df.shape 
print("Starting with", rows, "individuals and", cols, "features.")

Starting with 3861 individuals and 48 features.


In [157]:
# df = df.dropna(subset = ['Pop'])
# df['Pop'].value_counts(dropna=False)

In [158]:
#Turn pop to one hot.
# df['Pop'] = df['Pop'].map({'W':0, 'B':1, 'H':2})
# cat_col = pd.DataFrame(df['Pop'])
# from sklearn import preprocessing
# enc = preprocessing.OneHotEncoder()
# enc.fit(cat_col)
# cat_col_oh = pd.DataFrame(enc.transform(cat_col).toarray())
# del df['Pop']
# df = pd.concat([df, cat_col_oh], axis=1)

In [159]:
#Consider our target col, Sex.
print("Unique entries.")
df['Sex'].value_counts(dropna=False)

Unique entries.


M      2973
F       887
NaN       1
Name: Sex, dtype: int64

In [160]:
#Clean N and Nan entries
df = df.dropna(subset = ['Sex'])

#Move from object to float.
df['Sex'] = df['Sex'].map({'M':1.0, 'F':0.})
df.describe()

,Sex,FStat,CStat,MStat,claxln,claapd,clavrd,scapht,scapbr,humxln,...,tibxln,tibpeb,tibdeb,tibnfx,tibnft,tibcir,fibxln,fibmdm,calcxl,calcbr
count,3860.000000,894.000000,1033.000000,1211.000000,1635.000000,1570.000000,1568.000000,1639.000000,1705.000000,3572.000000,...,3495.000000,1602.000000,1648.000000,1739.000000,1721.000000,1498.000000,3304.000000,1522.000000,1447.000000,1391.000000
mean,0.770207,171.201342,168.998064,174.027250,150.682569,12.415924,10.190689,154.868212,103.936070,328.865062,...,385.788555,76.134207,50.015777,34.504313,24.496223,93.779039,377.056598,15.296978,82.938493,42.479511
std,0.420754,9.649151,9.381864,6.279055,11.852631,2.028384,1.794479,13.947257,8.539951,20.914442,...,28.831919,6.111199,4.387955,3.831515,3.029720,9.349371,26.095440,2.008818,6.177163,3.729444
min,0.000000,137.000000,134.000000,156.000000,118.000000,7.000000,4.000000,98.000000,83.000000,236.000000,...,268.000000,27.000000,38.000000,22.000000,17.000000,68.000000,276.000000,9.000000,65.000000,28.000000
25%,1.000000,165.000000,163.000000,170.000000,142.000000,11.000000,9.000000,144.000000,97.000000,315.000000,...,366.000000,71.000000,47.000000,32.000000,22.000000,87.000000,360.000000,14.000000,78.000000,40.000000
50%,1.000000,171.000000,169.000000,174.000000,151.000000,12.000000,10.000000,157.000000,104.000000,330.000000,...,386.000000,77.000000,50.000000,35.000000,24.000000,94.000000,377.000000,15.000000,83.000000,43.000000
75%,1.000000,178.000000,175.000000,178.000000,160.000000,14.000000,11.000000,165.000000,110.000000,343.000000,...,405.000000,81.000000,53.000000,37.000000,26.000000,100.000000,394.000000,17.000000,87.000000,45.000000
max,1.000000,202.000000,203.000000,191.000000,183.000000,22.000000,20.000000,199.000000,131.000000,404.000000,...,483.000000,94.000000,67.000000,55.000000,41.000000,127.000000,469.000000,22.000000,100.000000,83.000000


In [161]:
#Split into data and labels.
X = df.drop(['Sex'], axis=1)
y = df[['Sex']]

In [162]:
#Split train test
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01)

In [163]:
X_train.isnull().sum().sum(),X_train.isnull().sum().sum()

(94759, 94759)

In [164]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3821, 47), (39, 47), (3821, 1), (39, 1))

In [165]:
#Standardize df. Can't use scikit's scale because it doesn't like Nan.
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / (X_train_std)
#Intentionally using train stats for test set.
X_test = ( X_test - X_train_mean) / (X_train_std)
X_train.describe()

,FStat,CStat,MStat,claxln,claapd,clavrd,scapht,scapbr,humxln,humebr,...,tibxln,tibpeb,tibdeb,tibnfx,tibnft,tibcir,fibxln,fibmdm,calcxl,calcbr
count,8.890000e+02,1.023000e+03,1.200000e+03,1.619000e+03,1.556000e+03,1.554000e+03,1.621000e+03,1.687000e+03,3.535000e+03,1.749000e+03,...,3.459000e+03,1.586000e+03,1.635000e+03,1.722000e+03,1.705000e+03,1.488000e+03,3.273000e+03,1.510000e+03,1.437000e+03,1.381000e+03
mean,1.443914e-15,1.041200e-15,1.256958e-15,-4.952459e-16,-2.465894e-16,2.820209e-16,1.659513e-16,-4.593572e-17,1.186353e-15,-9.712071e-17,...,-3.085130e-16,-4.034052e-16,3.826206e-17,-9.051348e-16,-2.176167e-16,7.057527e-16,7.830922e-16,-6.896617e-17,-1.049111e-15,1.614687e-16
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-3.537062e+00,-3.735797e+00,-2.881785e+00,-2.759629e+00,-2.666208e+00,-3.447531e+00,-4.080878e+00,-2.453615e+00,-4.437715e+00,-2.914732e+00,...,-4.078939e+00,-8.026343e+00,-2.737226e+00,-3.259139e+00,-2.467409e+00,-2.752894e+00,-3.864475e+00,-3.139163e+00,-2.904786e+00,-3.876410e+00
25%,-6.400975e-01,-6.386059e-01,-6.453664e-01,-7.295620e-01,-6.956013e-01,-6.618284e-01,-7.775990e-01,-8.095183e-01,-6.604485e-01,-7.096454e-01,...,-6.853093e-01,-8.360387e-01,-6.859792e-01,-6.526754e-01,-8.206023e-01,-7.222155e-01,-6.505924e-01,-6.414625e-01,-7.987428e-01,-6.603966e-01
50%,-1.931931e-02,2.192369e-03,-6.389766e-03,3.171327e-02,-2.029497e-01,-1.046879e-01,1.559362e-01,1.253017e-02,5.675401e-02,1.723893e-01,...,7.268159e-03,1.444573e-01,-2.230373e-03,1.292636e-01,-1.618796e-01,2.592937e-02,-1.636563e-04,-1.419223e-01,1.127372e-02,1.436067e-01
75%,7.049219e-01,6.429906e-01,6.325869e-01,7.929885e-01,7.823536e-01,4.524526e-01,7.304195e-01,7.171431e-01,6.783295e-01,7.604124e-01,...,6.652168e-01,7.981212e-01,6.815185e-01,6.505563e-01,4.968430e-01,6.671964e-01,6.502651e-01,8.571579e-01,6.592869e-01,6.796089e-01
max,3.188035e+00,3.633382e+00,2.709261e+00,2.738470e+00,4.723567e+00,5.466717e+00,3.171973e+00,3.183289e+00,3.594953e+00,2.377476e+00,...,3.366269e+00,2.922529e+00,3.872346e+00,5.342191e+00,5.437263e+00,3.552898e+00,3.519804e+00,3.354859e+00,2.765330e+00,1.086365e+01


In [166]:
#The imputer. I haven't looked closely at this algo. Try other variations.
from fancyimpute import KNN, SoftImpute, NuclearNormMinimization, MatrixFactorization, BiScaler
if True:
#     X_train_filled = pd.DataFrame(KNN(k=1).complete(X_train))
#     X_test_filled = pd.DataFrame(KNN(k=1).complete(X_test))
    X_train_filled = SoftImpute().complete(X_train)
    X_test_filled = SoftImpute().complete(X_test)
    
else:
    from sklearn.preprocessing import Imputer
    imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
    X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
    # X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
    X_test_filled = pd.DataFrame(imp.transform(X_test_filled))

[SoftImpute] Max Singular Value of X_init = 199.921319
[SoftImpute] Iter 1: observed MAE=0.066640 rank=47
[SoftImpute] Iter 2: observed MAE=0.067265 rank=47
[SoftImpute] Iter 3: observed MAE=0.067835 rank=47
[SoftImpute] Iter 4: observed MAE=0.068323 rank=47
[SoftImpute] Iter 5: observed MAE=0.068673 rank=46
[SoftImpute] Iter 6: observed MAE=0.068895 rank=46
[SoftImpute] Iter 7: observed MAE=0.069083 rank=46
[SoftImpute] Iter 8: observed MAE=0.069237 rank=46
[SoftImpute] Iter 9: observed MAE=0.069361 rank=46
[SoftImpute] Iter 10: observed MAE=0.069463 rank=46
[SoftImpute] Iter 11: observed MAE=0.069545 rank=46
[SoftImpute] Iter 12: observed MAE=0.069612 rank=46
[SoftImpute] Iter 13: observed MAE=0.069667 rank=46
[SoftImpute] Iter 14: observed MAE=0.069713 rank=46
[SoftImpute] Iter 15: observed MAE=0.069752 rank=46
[SoftImpute] Iter 16: observed MAE=0.069784 rank=46
[SoftImpute] Iter 17: observed MAE=0.069810 rank=46
[SoftImpute] Iter 18: observed MAE=0.069833 rank=46
[SoftImpute] Iter 

In [167]:
#Num zero entries
(X_train_filled == 0).astype(int).sum().sum(),(X_test_filled == 0).astype(int).sum().sum()

(1692, 0)

In [168]:
#Impute zeros with mean. Could do this better.
#Think this introduces an improbable bug when the train set
#Has representation in a col that the test set doesn't or 
#Vice versa. Ignoring it for now. 
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values=0, strategy='mean', axis=0)
X_train_filled = pd.DataFrame(imp.fit_transform(X_train_filled))
# X_train_filled = pd.DataFrame(imp.transform(X_train_filled))
X_test_filled = pd.DataFrame(imp.transform(X_test_filled))

In [169]:
#No zero values
(X_train_filled == 0).astype(int).sum().sum(), (X_test_filled == 0).astype(int).sum().sum()

(0, 0)

In [170]:
#Notice that the training (and testing) set is no longer standardized
X_train_filled.describe()

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
count,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,...,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000
mean,0.022591,0.048783,-0.106478,0.036503,0.015081,0.006759,0.030285,0.035092,-0.007377,0.017911,...,-0.011019,0.013182,0.009214,0.021442,0.021055,0.019792,-0.005648,0.002060,0.022080,0.012263
std,0.693122,0.752153,0.671208,0.704942,0.669746,0.655164,0.697175,0.713221,0.977388,0.702934,...,0.972690,0.692646,0.680736,0.697552,0.690695,0.691849,0.968505,0.644849,0.688484,0.659888
min,-3.537062,-3.735797,-2.881785,-2.759629,-2.666208,-3.447531,-4.080878,-2.453615,-4.437715,-2.914732,...,-4.078939,-8.026343,-2.737226,-3.259139,-2.467409,-2.752894,-4.263434,-3.139163,-2.904786,-3.876410
25%,-0.329708,-0.364269,-0.485622,-0.252150,-0.202950,-0.104688,-0.204822,-0.232716,-0.660448,-0.206239,...,-0.650680,-0.182375,-0.192845,-0.190437,-0.165223,-0.225178,-0.612332,-0.141922,-0.290770,-0.225651
50%,0.078276,0.108992,-0.137903,0.065195,0.032703,0.006759,0.070399,0.074473,0.015449,0.055288,...,-0.009846,0.049666,0.022258,0.048107,0.041973,0.043853,-0.000164,0.008316,0.054821,0.047944
75%,0.418154,0.509173,0.207481,0.370058,0.285569,0.120110,0.360212,0.364837,0.630516,0.319395,...,0.630588,0.307873,0.225686,0.258818,0.214714,0.281761,0.612005,0.118456,0.335280,0.261669
max,3.188035,3.633382,2.709261,2.738470,4.723567,5.466717,3.171973,3.183289,3.594953,2.377476,...,3.366269,2.922529,3.872346,5.342191,5.437263,3.552898,3.519804,3.354859,2.765330,10.863651


In [171]:
#Should we standardize again?
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train_filled)
X_train_filled = pd.DataFrame(scaler.transform(X_train_filled))
#Uses test stats
X_test_filled = pd.DataFrame(scaler.transform(X_test_filled))

In [172]:
#Save to disk
X_train_filled.to_csv("data/X_train_preproc.csv", index=False)
#Ravel strips col name and reindexes
pd.DataFrame(np.ravel(y_train)).to_csv("data/y_train_preproc.csv", index=False)

X_test_filled.to_csv("data/X_test_preproc.csv", index=False)
pd.DataFrame(np.ravel(y_test)).to_csv("data/y_test_preproc.csv", index=False)

In [173]:
X_test_filled

,0,1,2,3,4,5,6,7,8,9,...,37,38,39,40,41,42,43,44,45,46
0,0.792725,1.853595,0.430666,1.673288,-1.061261,3.231852,2.034530,1.285775,1.631257,1.683905,...,0.980160,2.077240,1.322596,0.902006,1.165861,1.090384,1.309489,1.326217,1.631649,2.636173
1,0.145227,0.867638,1.577293,0.408764,0.018676,0.233235,0.152046,0.163335,0.946292,0.285896,...,1.051372,0.257343,0.305150,0.491054,0.273093,0.529431,0.953901,0.726978,0.785837,0.223101
2,-1.082649,-1.796835,-0.805779,-1.446866,-2.532614,-1.871117,-2.807194,-1.678405,-1.304307,-2.290119,...,-1.263006,-1.462175,-1.691077,-2.087654,-1.695644,-1.446416,-1.219142,-2.316716,-1.462617,-2.033202
3,0.984559,0.637474,1.207177,-0.231065,1.881444,-0.170127,1.622469,0.791745,0.652736,0.428950,...,1.443035,0.661474,-0.686519,3.144252,1.642779,2.480898,1.507039,2.875743,1.396314,1.011435
4,-1.254798,-0.253743,0.300261,1.553282,-1.061261,-2.721611,-3.013225,-1.678405,0.370233,-0.821224,...,-0.260047,-0.965069,-0.262443,-0.278746,-1.027453,-0.458510,-0.206782,-0.588188,-0.551651,-0.852647
5,0.188550,0.203059,0.508308,3.113359,1.145768,1.530862,-0.025778,2.109158,1.826962,-2.917596,...,-0.119785,0.352446,0.389161,-0.129799,0.249748,0.253884,-0.297536,0.553643,0.077064,0.466821
6,-0.396491,-1.072704,0.243569,-1.806884,-1.184819,-0.398716,-0.746886,-1.843081,-0.815047,-0.198527,...,-0.077006,-1.226214,-0.589934,-1.713947,-0.264891,-0.779725,-0.028367,-0.710735,-0.768224,-0.902651
7,-0.239854,0.648102,-0.096206,-0.076266,-0.118508,-0.156495,-0.350314,-0.229255,-0.130081,-0.127842,...,-0.622102,0.062401,0.108086,-0.370784,-0.003001,-0.094506,-0.310415,-0.491967,0.047621,-0.067761
8,0.318811,0.620566,-0.709048,-0.246807,0.615342,1.540914,1.313423,0.133038,-0.032229,1.474745,...,-0.194832,1.133396,0.479732,0.528299,1.165861,0.394693,-0.270906,0.911205,0.424524,1.544334
9,0.536688,0.231781,-0.095984,2.393324,-0.325585,0.680368,0.077238,2.767865,0.359179,0.638109,...,0.303649,0.189552,0.987744,-0.219116,0.212026,0.119429,0.479782,0.551454,0.523997,0.518765
